<a href="https://colab.research.google.com/github/LeeDongN/Medica_AI2/blob/main/final_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from matplotlib import pyplot as plt
import os
import sklearn
from google.colab import drive
from scipy.signal import find_peaks
drive.mount('/content/drive')
path = '/content/drive/My Drive/data/'

Mounted at /content/drive


# first data import

In [2]:
first_data = []
first_data = pd.read_csv('/content/drive/My Drive/data2/REF.CSV', skiprows=24)
first_data_origin = pd.read_csv('/content/drive/My Drive/data2/REF.CSV', skiprows=24)
first_a = float(first_data_origin.columns[0])
first_b = float(first_data_origin.columns[1])
first_data.loc[0] = [first_a, first_b]
first_data.columns=['Wave_length', 'Amplitude']
first_data['label']=1.3
first_np = first_data[['Wave_length', 'Amplitude']].to_numpy()
first_np = np.ravel(first_np, order='F')
first_np = np.array([first_np])

# label

In [ ]:
first_np[0]

array([1.4750000e+03, 1.4750397e+03, 1.4750595e+03, ..., 1.3470000e-06,
       1.2690000e-06, 1.4060000e-06])

# functions

#### importing first data function

In [4]:
def first_data_import():
  # import the first_data
  # first_data making
  first_data = []
  first_data = pd.read_csv('/content/drive/My Drive/data2/REF.CSV', skiprows=24)
  first_data_origin = pd.read_csv('/content/drive/My Drive/data2/REF.CSV', skiprows=24)
  first_a = float(first_data_origin.columns[0])
  first_b = float(first_data_origin.columns[1])
  first_data.loc[0] = [first_a, first_b]
  first_data.columns=['Wave_length', 'Amplitude']
  first_data['label']=1.3
  first_np = first_data[['Wave_length', 'Amplitude']].to_numpy()
  first_np = np.ravel(first_np, order='F')
  first_np = np.array([first_np])

  return first_data, first_np

#### importing whole data and stack function

In [5]:
def stack_data(path):
  # the array that combined whole data
  stack_np = np.array([[]])
  # extract the folder name (it will be the label)
  folder_list = os.listdir(path)
  # extract each folder name
  for folder_name in folder_list:
    temp_folder_list = os.listdir(path + folder_name)
    # extract each file name
    for file_name in temp_folder_list:
      # Read the data into a Pandas DataFrame
      data = pd.read_csv(path + folder_name + '/' + file_name, skiprows=24)
      data_origin = pd.read_csv(path + folder_name + '/' + file_name, skiprows=24)
      # generate columns name and restore the removed data(the first data)
      a = float(data_origin.columns[0])
      b = float(data_origin.columns[1])
      data.loc[0] = [a, b]
      data.columns=['Wave_length', 'Amplitude']
      # make the pandas data to numpy
      temp = data[['Wave_length', 'Amplitude']].to_numpy()
      # make it flat
      temp = np.ravel(temp, order='F')
      temp = np.array([temp])
      stack_np = np.concatenate((first_np, temp), axis = 1)
  return stack_np

#### removing DC function

In [6]:
# Implementing a simple DC removal function
def remove_dc(signal):
    # Calculate the mean of the signal
    signal_mean = np.mean(signal)
    # Remove the mean from the signal
    dc_removed_signal = signal - signal_mean
    return dc_removed_signal

#### finding peaks function

In [7]:
# Find peaks
def detect_peaks(signal, threshold=None, min_distance=1, prominence=None):
    if threshold is None:
        # Set threshold as a percentage of the maximum peak
        max_peak = np.max(signal)
    peaks, _ = find_peaks(signal, height=threshold, distance=min_distance, prominence=prominence)
    return peaks

#### peak values extracting function

In [8]:
def extract_peaks(X, Y, peak_range):
  # pandas to numpy
  peak_X = X.to_numpy()
  peak_Y = Y.to_numpy()
  # extract the peak value index
  index = list(peak_Y).index(max(peak_Y))

  # temp saving
  extracted_x = []
  extracted_y = []

  # extract the values from highest peak, and surrounding peaks
  for k in range(peak_range):
    # index labeling
    i = k + 1
    p_temp_index = index + i
    m_temp_index = index - i
    # extract y
    extracted_y.append(peak_Y[p_temp_index])
    extracted_y.append(peak_Y[m_temp_index])
    # extract X
    extracted_x.append(peak_X[p_temp_index])
    extracted_x.append(peak_X[m_temp_index])

  # extrack the center(highest) value
  extracted_x.append(peak_X[index])
  extracted_y.append(peak_Y[index])

  return extracted_x, extracted_y

#### final function

In [44]:
def final_function(path, peak_range):
  # extracted peak amplitudes and wavelength array
  extracted_data = []
  # the array that combined whole data
  stack_np = np.array([[]])
  # extract the folder name (it will be the label)
  folder_list = os.listdir(path)
  # extract each folder name
  for folder_name in folder_list:
    temp_folder_list = os.listdir(path + folder_name)
    # extract each file name
    for file_name in temp_folder_list:
      # Read the data into a Pandas DataFrame
      data = pd.read_csv(path + folder_name + '/' + file_name, skiprows=24)
      data_origin = pd.read_csv(path + folder_name + '/' + file_name, skiprows=24)
      # generate columns name and restore the removed data(the first data)
      a = float(data_origin.columns[0])
      b = float(data_origin.columns[1])
      data.loc[0] = [a, b]
      data.columns=['Wave_length', 'Amplitude']

      # DC removed
      dc_removed_Y = remove_dc(data['Amplitude'])

      # dececting peaks
      min_distance = 100  # Set the minimum distance between peaks
      local_maxima_indices = detect_peaks(dc_removed_Y, min_distance=min_distance)
      local_maxima_x = data['Wave_length'][local_maxima_indices]
      local_maxima_y = dc_removed_Y[local_maxima_indices]

      # extract_peaks
      extracted_x, extracted_y = extract_peaks(local_maxima_x, local_maxima_y, peak_range)
      # saving
      temp = []
      temp.extend(extracted_x)
      temp.extend(extracted_y)
      extracted_data.append(temp)

  extracted_np_data = np.array(extracted_data)
  # data = [wavelength, amplitude] -> the number of each parameter is related with the peak range
  return extracted_np_data

# Test

In [43]:
test = final_function(path, 3)
print(test.shape)

(252, 14)
